In [5]:
# ===============================
# SUPERMARKET SALES DATA CLEANING  
# ===============================

import pandas as pd
import numpy as np

# -------------------------------
# 1. Load Raw Data
# -------------------------------
df = pd.read_csv("C:\\Users\\Sakshi Narawade\\Documents\\BA\\Project 1\\data\\raw data\\SuperMarket Analysis.csv")

print("Initial Shape:", df.shape)
display(df.head())

# -------------------------------
# 2. Standardize Column Names
# -------------------------------
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace("%", "pct")
)

print("\nStandardized Columns:")
print(df.columns)

# -------------------------------
# 3. Remove Duplicates
# -------------------------------
df = df.drop_duplicates()
print("\nShape after removing duplicates:", df.shape)

# -------------------------------
# 4. Handle Missing Values
# -------------------------------
numeric_cols = [
    "unit_price", "quantity", "tax_5pct", "sales",
    "cogs", "gross_margin_percentage", "gross_income", "rating"
]
df[numeric_cols] = df[numeric_cols].fillna(0)

categorical_cols = [
    "branch", "city", "customer_type",
    "gender", "product_line", "payment"
]
df[categorical_cols] = df[categorical_cols].fillna("Unknown")

# -------------------------------
# 5. Date & Time Processing (FIXED)
# -------------------------------
df["date"] = df["date"].astype(str).str.strip()

def parse_mixed_date(date_str):
    if date_str in ["", "nan", "NaT"]:
        return pd.NaT
    for fmt in ("%d-%m-%Y", "%d/%m/%Y", "%m/%d/%Y", "%Y-%m-%d"):
        try:
            return pd.to_datetime(date_str, format=fmt)
        except ValueError:
            continue
    return pd.to_datetime(date_str, errors="coerce")

df["date"] = df["date"].apply(parse_mixed_date)

# Standardize format
df["date"] = df["date"].dt.strftime("%d/%m/%y")

# Extract features safely
df["_date_parsed"] = pd.to_datetime(df["date"], format="%d/%m/%y")

df["time"] = pd.to_datetime(df["time"], errors="coerce").dt.time
df["year"] = df["_date_parsed"].dt.year
df["month"] = df["_date_parsed"].dt.month
df["day"] = df["_date_parsed"].dt.day
df["weekday"] = df["_date_parsed"].dt.day_name()

df.drop(columns=["_date_parsed"], inplace=True)

# -------------------------------
# 6. Business Metric Engineering
# -------------------------------
df["sales_velocity"] = df["sales"] / df["quantity"]
df["avg_transaction_value"] = df["sales"]
df["profit_margin_pct"] = (df["gross_income"] / df["sales"]) * 100
df["high_value_txn"] = np.where(df["sales"] > df["sales"].mean(), "Yes", "No")

# -------------------------------
# 7. Standardize Text Fields
# -------------------------------
df["city"] = df["city"].str.title()
df["branch"] = df["branch"].str.upper()
df["product_line"] = df["product_line"].str.title()
df["payment"] = df["payment"].str.title()

# -------------------------------
# 8. Final Data Validation
# -------------------------------
print("\nFinal Dataset Info:")
df.info()
display(df.head())

# -------------------------------
# 9. Export Cleaned Data
# -------------------------------
df.to_csv(
    r"C:\Users\Sakshi Narawade\Documents\BA\Project 1\data\processed\supermarket_sales_cleaned.csv",
    index=False
)

print("\n✅ Cleaned data saved as: supermarket_sales_cleaned.csv")


Initial Shape: (1000, 17)


,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Sales,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,Alex,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,01-05-2019,1:08:00 PM,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,Giza,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,03-08-2019,10:29:00 AM,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,Alex,Yangon,Normal,Female,Home and lifestyle,46.33,7,16.2155,340.5255,03-03-2019,1:23:00 PM,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,Alex,Yangon,Member,Female,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,8:33:00 PM,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,Alex,Yangon,Member,Female,Sports and travel,86.31,7,30.2085,634.3785,02-08-2019,10:37:00 AM,Ewallet,604.17,4.761905,30.2085,5.3



Standardized Columns:
Index(['invoice_id', 'branch', 'city', 'customer_type', 'gender',
       'product_line', 'unit_price', 'quantity', 'tax_5pct', 'sales', 'date',
       'time', 'payment', 'cogs', 'gross_margin_percentage', 'gross_income',
       'rating'],
      dtype='object')

Shape after removing duplicates: (1000, 17)

Final Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   invoice_id               1000 non-null   object 
 1   branch                   1000 non-null   object 
 2   city                     1000 non-null   object 
 3   customer_type            1000 non-null   object 
 4   gender                   1000 non-null   object 
 5   product_line             1000 non-null   object 
 6   unit_price               1000 non-null   float64
 7   quantity                 1000 non-null   int64  
 8  

C:\Users\Sakshi Narawade\AppData\Local\Temp\ipykernel_100764\2323294152.py:74: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["time"] = pd.to_datetime(df["time"], errors="coerce").dt.time


,invoice_id,branch,city,customer_type,gender,product_line,unit_price,quantity,tax_5pct,sales,...,gross_income,rating,year,month,day,weekday,sales_velocity,avg_transaction_value,profit_margin_pct,high_value_txn
0,750-67-8428,ALEX,Yangon,Member,Female,Health And Beauty,74.69,7,26.1415,548.9715,...,26.1415,9.1,2019,5,1,Wednesday,78.4245,548.9715,4.761905,Yes
1,226-31-3081,GIZA,Naypyitaw,Normal,Female,Electronic Accessories,15.28,5,3.8200,80.2200,...,3.8200,9.6,2019,8,3,Saturday,16.0440,80.2200,4.761905,No
2,631-41-3108,ALEX,Yangon,Normal,Female,Home And Lifestyle,46.33,7,16.2155,340.5255,...,16.2155,7.4,2019,3,3,Sunday,48.6465,340.5255,4.761905,Yes
3,123-19-1176,ALEX,Yangon,Member,Female,Health And Beauty,58.22,8,23.2880,489.0480,...,23.2880,8.4,2019,1,27,Sunday,61.1310,489.0480,4.761905,Yes
4,373-73-7910,ALEX,Yangon,Member,Female,Sports And Travel,86.31,7,30.2085,634.3785,...,30.2085,5.3,2019,8,2,Friday,90.6255,634.3785,4.761905,Yes



✅ Cleaned data saved as: supermarket_sales_cleaned.csv
